In [106]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
import os
import random


In [107]:
IMAGE_SIZE = [224, 224]
random.seed(42)

In [108]:
# initializing VGG with its own weights,
# include_top false means it removes top dense layers which are optimized for VGG images 
# we need to add new layers and train with our data.
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False) # here [3] is 3 channels Red blue Green


In [109]:
for layer in vgg.layers:
  layer.trainable = False
#Here we Froze the weights of VGG- Meaning we are using existing features from VGG adn mixing with our dense layers

In [110]:
# Model without any dense layers, features only from VGNET and direct predictions.
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
prediction = Dense(5, activation='softmax')(x) # here we have 5 output classes so i gave 5 in output layer


In [111]:
# Creating VGNET model
model = Model(inputs=vgg.input, outputs=prediction)

In [112]:
model.summary()


Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0  

In [113]:

model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.RMSprop(),
  metrics=['accuracy']
)


In [114]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [115]:
#Input needs to be (224,224)
# we are addign data augmentation to zoom in out , and flipping horizontally
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)# add augments with horizontal flip

test_datagen = ImageDataGenerator(rescale = 1./255)#divide by 255 to normalize/scale

training_set = train_datagen.flow_from_directory('SecondDataset/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('SecondDataset/Test',
                                           target_size = (224, 224),
                                             batch_size = 32,
                                             class_mode = 'categorical')

Found 214 images belonging to 5 classes.
Found 81 images belonging to 5 classes.


In [116]:
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/5
7/7 [==============================] - 8s 1s/step - loss: 4.9197 - accuracy: 0.2336 - val_loss: 2.8664 - val_accuracy: 0.3827
Epoch 2/5
7/7 [==============================] - 7s 929ms/step - loss: 2.0409 - accuracy: 0.4533 - val_loss: 0.7690 - val_accuracy: 0.7037
Epoch 3/5
7/7 [==============================] - 7s 932ms/step - loss: 1.5301 - accuracy: 0.4579 - val_loss: 1.4758 - val_accuracy: 0.4691
Epoch 4/5
7/7 [==============================] - 7s 953ms/step - loss: 1.2293 - accuracy: 0.5935 - val_loss: 0.9201 - val_accuracy: 0.5802
Epoch 5/5
7/7 [==============================] - 7s 969ms/step - loss: 1.0365 - accuracy: 0.6776 - val_loss: 0.6704 - val_accuracy: 0.7407


In [117]:
#This did not work very will with VG net architecture.
#Transfer learning may not always work. we need to Tweak some parameters and try it agian,
score = model.evaluate(test_set)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

3/3 [==============================] - 2s 492ms/step - loss: 0.6704 - accuracy: 0.7407
Test Loss: 0.6704394817352295
Test accuracy: 0.7407407760620117


In [118]:
# i am adding more train layers to customize the model more.



In [119]:
#Second Try

In [120]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


In [121]:
for layer in vgg.layers:
  layer.trainable = False

In [122]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
x=Dense(100, activation='relu')(x)
Dropout(0.5)(x)
x=Dense(100, activation='relu')(x)
prediction = Dense(5, activation='softmax')(x)


In [123]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [124]:
model.summary()


Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0  

In [125]:
from tensorflow.keras.callbacks import EarlyStopping
model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.RMSprop(),
  metrics=['accuracy']
)
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
callback = [earlystop]


In [126]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=8,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),callbacks=callback
)


Epoch 1/8
7/7 [==============================] - 7s 978ms/step - loss: 5.6932 - accuracy: 0.2009 - val_loss: 1.4766 - val_accuracy: 0.4321
Epoch 2/8
7/7 [==============================] - 6s 892ms/step - loss: 1.9636 - accuracy: 0.2757 - val_loss: 1.4225 - val_accuracy: 0.4074
Epoch 3/8
7/7 [==============================] - 6s 892ms/step - loss: 1.3922 - accuracy: 0.4252 - val_loss: 1.4853 - val_accuracy: 0.6173
Epoch 4/8
7/7 [==============================] - 6s 871ms/step - loss: 1.3472 - accuracy: 0.4953 - val_loss: 1.9951 - val_accuracy: 0.2346
Epoch 5/8
7/7 [==============================] - 6s 875ms/step - loss: 1.2555 - accuracy: 0.4533 - val_loss: 1.7040 - val_accuracy: 0.3827
Epoch 6/8
7/7 [==============================] - 6s 872ms/step - loss: 1.2323 - accuracy: 0.5234 - val_loss: 0.9181 - val_accuracy: 0.5679
Epoch 7/8
7/7 [==============================] - 6s 845ms/step - loss: 0.8553 - accuracy: 0.6729 - val_loss: 0.7792 - val_accuracy: 0.7901
Epoch 8/8
7/7 [============

In [127]:
#adding more layers which are trainablle did not help our model.We need to try a different approach.

In [128]:
score = model.evaluate(test_set)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

3/3 [==============================] - 2s 448ms/step - loss: 0.7195 - accuracy: 0.7160
Test Loss: 0.7194870710372925
Test accuracy: 0.7160493731498718


In [129]:
#Thrid Model - Now i will only use one dense layer and one for output.


In [130]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
for layer in vgg.layers:
  layer.trainable = False

In [131]:
# our layers - you can add more if you want
x = Flatten()(vgg.output)
# x = Dense(1000, activation='relu')(x)
x=Dense(50, activation='relu')(x)
prediction = Dense(5, activation='softmax')(x)


In [132]:
from tensorflow.keras.callbacks import EarlyStopping
model.compile(
  loss='categorical_crossentropy',
  optimizer=tf.keras.optimizers.RMSprop(),
  metrics=['accuracy']
)
earlystop = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
callback = [earlystop]


In [133]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=8,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),callbacks=callback
)


Epoch 1/8
7/7 [==============================] - 7s 990ms/step - loss: 1.7249 - accuracy: 0.5421 - val_loss: 1.1603 - val_accuracy: 0.5556
Epoch 2/8
7/7 [==============================] - 6s 883ms/step - loss: 1.1930 - accuracy: 0.5561 - val_loss: 0.4561 - val_accuracy: 0.8765
Epoch 3/8
7/7 [==============================] - 6s 852ms/step - loss: 0.6041 - accuracy: 0.7523 - val_loss: 0.8231 - val_accuracy: 0.5926
Epoch 4/8
7/7 [==============================] - 6s 846ms/step - loss: 0.6869 - accuracy: 0.6822 - val_loss: 1.0543 - val_accuracy: 0.6173
Epoch 5/8
7/7 [==============================] - 6s 848ms/step - loss: 0.6117 - accuracy: 0.7570 - val_loss: 0.2377 - val_accuracy: 0.9506
Epoch 6/8
7/7 [==============================] - 6s 871ms/step - loss: 0.5032 - accuracy: 0.7850 - val_loss: 0.3464 - val_accuracy: 0.8642
Epoch 7/8
7/7 [==============================] - 6s 880ms/step - loss: 0.6883 - accuracy: 0.7850 - val_loss: 0.3869 - val_accuracy: 0.8148
Epoch 8/8
7/7 [============

In [134]:
score = model.evaluate(test_set)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

3/3 [==============================] - 2s 467ms/step - loss: 0.2019 - accuracy: 0.9630
Test Loss: 0.20189321041107178
Test accuracy: 0.9629629850387573


In [135]:
#This model Fitted Best compared to the model whit direct predictions of our data with VGG 
#it worked better than the one with 2 dense layers.

In [136]:
# Its good to use simple models than complex models with more features.

In [137]:
# Here we explored the Power of transfer learning, by using existing features from VGG
# and adding only one dense layer other than output we saved a lot of compute power and time and
#achieved great accuracy.